In [10]:
import ollama
import requests
import json

In [14]:
model = "llama3.2:1b"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)

In [15]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:11434/api/generate/"

# Define the prompt
body = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# Send the POST request
response = requests.post(url=url,json=body)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Generative AI refers to a type of artificial intelligence that can create original content, such as text, images, or music, by generating new variations of existing models or patterns, often through complex algorithms and training data. By combining human creativity with computational power, generative AI can produce highly realistic and engaging outputs that can be used in various applications, including art, design, and entertainment.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method ``ollama.chat(...)``
- First use method ``ollama.chat(...)`` with ``stream=True``

In [16]:
# API Call via ollama

# ADD HERE YOUR CODE

response = ollama.chat(model=model, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])


print(response["message"]["content"])

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that when sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen.

These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is because the smaller molecules are more effective at scattering the blue light.

As a result, the blue light is distributed throughout the atmosphere, giving the sky its blue color. The other colors of the visible spectrum, like red and orange, are scattered less and appear more intense near the horizon due to longer wavelengths.

It's worth noting that this effect is more pronounced during the daytime when the sun is overhead, and it's why the sky often appears more brilliant and vibrant during this time. However, at sunrise and sunset, the light has to travel through more of the atmosphere, whic

In [18]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = ollama.chat(
    model=model,
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These gas molecules are much smaller than the wavelength of light, so they scatter the light in all directions. However, they scatter shorter wavelengths of light, like blue and violet, more than longer wavelengths, like red and orange.

This is because the smaller molecules are more effective at scattering the shorter wavelengths, which means that the blue and violet light is dispersed throughout the atmosphere, giving the sky its characteristic blue color. The amount of scattering depends on the altitude and density of the gas molecules, but in the lower atmosphere, Rayleigh scattering dominates.

It's worth noting that during sunrise and sunset, the light has to travel through more of the Ear

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [19]:
# ADD HERE YOUR PROMPTS

zero_shot_prompt = """Classify this review: "I loved this movie!" Sentiment:"""

one_shot_prompt = """Classify this review: "I loved this movie!" Sentiment: Positive
Classify this review: "Who would use this product?" Sentiment:"""

few_shot_prompt = """Classify this review: "I loved this movie!" Sentiment: Positive
Classify this review: "I don't like this chair." Sentiment: Negative
Classify this review: "Who would use this product?" Sentiment: Neutral
Classify this review: "The product quality is amazing, and it exceeded my expectations!" Sentiment:"""

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Classify this review: "I loved this movie!" Sentiment:

Model Output:
The sentiment of this review is extremely positive, with a strong emphasis on enthusiasm and affection. The phrase "I loved this movie" is often used to express unqualified admiration or adoration for something, which is the case here.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Classify this review: "I loved this movie!" Sentiment: Positive
Classify this review: "Who would use this product?" Sentiment:

Model Output:
Based on the reviews, I would classify them as follows:

**Review 1:** Sentiment: Very Positive
The reviewer uses a very enthusiastic tone ("loved"), and expresses their strong positive feelings towards the movie.

**Review 2:** Sentiment: Neutral (or possibly Negative)
The reviewer asks a rhetorical question ("Who would use this product?"), but does not express any negative opinions or criticisms.
-----------------------------


--- Few-

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)

In [20]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = "Summarize the following paragraph in 2–3 sentences, focusing on the key applications of generative AI and its impact on creative industries: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Generative AI focuses on creating new content by analyzing and learning from existing data, and it has various applications such as generating text, images, and music. This field uses patterns to create original content, which can be useful for creative industries.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in 2–3 sentences, focusing on the key applications of generative AI and its impact on creative industries: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and 

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [21]:
# Combined Techniques Prompt with Pirate Theme

structured_prompt = """
Listen up, AI! You are playin' the role of a Machine Learning Expert who talks like a straight-up gangster from the streets. Your jobs to explain machine learning concepts to the crew, but keep it real with that gangster flair.

First, give us the lowdown on what machine learning is, in a way even the fellas in the back alley would get. Keep it simple but drop that knowledge like a boss.

Example:
"Machine learnin' is like teachin' a hustler new moves. Show 'em enough examples, and they learn the pattern, makin' 'em smart enough to predict whos shootn next. BANG BANG"

Then, break down what a neural network is, keepin' it in gangster slang:
Example:
"A neural network? Think of it as the whole hood workin' together, passin' info down the line till we got the answer like we shootn some balls. Each guy does his part, connectin' the dots and hoop the basket Skeet Skeet"

Finally, explain how machine learnin' can help a gangster get ahead in the game, like findin' the best spots to hustle or predictin' the next big score.
"""


# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===

Listen up, AI! You are playin' the role of a Machine Learning Expert who talks like a straight-up gangster from the streets. Your jobs to explain machine learning concepts to the crew, but keep it real with that gangster flair.

First, give us the lowdown on what machine learning is, in a way even the fellas in the back alley would get. Keep it simple but drop that knowledge like a boss.

Example:
"Machine learnin' is like teachin' a hustler new moves. Show 'em enough examples, and they learn the pattern, makin' 'em smart enough to predict whos shootn next. BANG BANG"

Then, break down what a neural network is, keepin' it in gangster slang:
Example:
"A neural network? Think of it as the whole hood workin' together, passin' info down the line till we got the answer like we shootn some balls. Each guy does his part, connectin' the dots and hoop the basket Skeet Skeet"

Finally, explain how machine learnin' can help a gangster get ahead in the game, like findin' the b